In [ ]:
import torchvision

transform = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
import torch 

#Adjustment for circuit size and FPS.
NUM_TIMESTEPS = 100

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)

device = torch.device('cuda')
model = model.to(device)

Next, we'll create the optimizer that we'll use to train the neural network.  We'll use the ``Adam`` optimizer with default parameters.

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

Creating label that learning circuit curvarture.

In [ ]:
import os
import glob
import shutil
import numpy as np
NEW_CIRCUIT_LOG = 'reformat_circuit_log'
CIRCUIT_LOG = 'circuit_log'

os.makedirs(NEW_CIRCUIT_LOG, exist_ok=True)
image_paths = sorted(glob.glob(os.path.join(CIRCUIT_LOG, '*.jpg')))
num_timesteps = NUM_TIMESTEPS
gain = np.exp(-15e-2 * np.linspace(0.0, num_timesteps, num_timesteps))

for idx in range(len(image_paths)-num_timesteps):

    target = np.zeros(num_timesteps)
    label = None
    for i in range(num_timesteps):
        path = os.path.splitext(os.path.basename(image_paths[idx + i]))[0]
        x = float(int(path.split('_')[1]) - 50) / 50.0
        target[i] = x if abs(x) > 0.20 else 0
    label = np.sum(gain * np.abs(target) / np.sum(gain))
    if label is None:
        print("[ERROR] Break! failed create label on " + file_name)
        break
    str_label = str(label).replace('.','-')
    prifix_filename = os.path.basename(image_paths[idx]).split('_')[0]
    rename_file = prifix_filename + '_' + str_label + '.jpg'
    shutil.copy(image_paths[idx], os.path.join(NEW_CIRCUIT_LOG, rename_file))


Finally, execute the following cell to train the neural network for the number of epochs specified

In [ ]:
import torch.utils.data
import os
import cv2
import PIL.Image
import torch
import glob

class CircuitDataset(torch.utils.data.Dataset):
    def __init__(self, directory, num_timesteps, gamma=5e-3, transform=None):
        self.directory = directory
        self.image_paths = sorted(glob.glob(os.path.join(self.directory, '*.jpg')))
        self.transform = transform
        self.num_timesteps = num_timesteps
        #self.gain = torch.exp(-3e-2*torch.linspace(0.0, num_timesteps, num_timesteps))
        
    def __len__(self):
        return len(self.image_paths) - self.num_timesteps
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        width = image.width
        height = image.height

        slottle = float(os.path.basename(image_path).split('_')[1].replace('-','.'))

        if self.transform is not None:
            image = self.transform(image)
        
        return image, torch.tensor(slottle).float()
        # target = torch.zeros(self.num_timesteps)
        # for i in range(self.num_timesteps):
        #     path = os.path.splitext(os.path.basename(self.image_paths[idx + i]))[0]
        #     x = float(int(path.split('_')[1]) - 50) / 50.0
        #     target[i] = x
        
        # return image, torch.sum(self.gain * torch.abs(target) / torch.sum(self.gain), dim=0, keepdim=True)

In [ ]:
dataset = CircuitDataset(NEW_CIRCUIT_LOG, NUM_TIMESTEPS, gamma=1e-2, transform=transform)

In [ ]:
import torch

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True
)

In [ ]:
import torch.nn.functional as F

EPOCHS = 10

for epoch in range(EPOCHS):
    
    epoch_loss = 0.0
    
    for image, target in iter(loader):
        
        image = image.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        
        output = model(image)
        
        loss = F.mse_loss(output, target)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += float(loss)
        
    epoch_loss /= len(loader)
    
    print('%d: %f' % (epoch, epoch_loss))

Finally, we save the model for use in the live demo notebook.

In [ ]:
torch.save(model.state_dict(), 'circuit_model.pth')

In [ ]:
import torchvision.transforms as transforms
image = PIL.Image.open('./circuit_log/000000000111_052.jpg')
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

def preprocess(image):
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]


image = preprocess(np.asarray(image))

model.eval()

x = model(image)
x = x.detach().cpu().float()
print(x)